In [1]:
import pandas as pd 
import numpy  as np
from wordfreq import word_frequency ,top_n_list,get_frequency_dict

from symspellpy import SymSpell, Verbosity
from itertools import islice

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from random import random, choice
import random
import time

import os
import re
from tqdm import tqdm  
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True,nb_workers=4)


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
from bnunicodenormalizer import Normalizer 
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()
bnorm=Normalizer()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
from typing import Dict, List, Tuple, Any, Union


import json

from datasets import load_metric

from tqdm.auto import tqdm
from IPython.display import display, Audio, HTML

cer = load_metric("cer")
wer = load_metric("wer")
import warnings 
warnings.filterwarnings('ignore')

In [4]:
def remove_punctuations(my_str):
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’…,<>.‚/?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None]) 

def wer(ref, hyp ,debug=False):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]

    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3

    DEL_PENALTY=1 # Tact
    INS_PENALTY=1 # Tact
    SUB_PENALTY=1 # Tact
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL

    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1

                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL

    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        lines = []
        compares = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("C\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("S\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("I\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("D\t" + r[i]+"\t"+"****")
    return lines

def error_label(ref,hyp):
    li=[]
    lines= wer(ref, hyp,debug=True)
    for line in reversed(lines):
        line = re.sub(r"\t"," ",line)
        line = line.split()
        li.append(line[0])
    return li

def wer_calc(label):
    s=label.count('S')
    d=label.count('D')
    i=label.count('I')
    return int((s+d+i)/len(label)*100)

# Common_voice

In [5]:
df_cv_1 = pd.read_csv('asr_bangla/data/cv_100k_p1.csv')
df_cv_2 = pd.read_csv('asr_bangla/data/cv_100k_p2.csv')
df_cv_3 = pd.read_csv('asr_bangla/data/symspell_nov.csv')
df_cv = pd.read_csv('cv_train.csv')

In [6]:
df_cv_3 = df_cv_3[(df_cv_3.source == 'train') & (df_cv_3.audio == 'noisy')]

In [7]:
df_cv_3 = df_cv_3[['path','wav2vec2','4gram']]

In [8]:
df_cv = df_cv[['path','sentence']]

In [9]:
df_cv_1= df_cv_1.append([df_cv_2,df_cv_3])
df_cv_1 = df_cv_1.reset_index(drop=True)

In [10]:
df_cv_1['path'] = df_cv_1.path.apply(lambda x: os.path.basename(str(x)))

In [11]:
df_cv = df_cv_1.merge(df_cv,on='path',how='inner')

In [12]:
df_cv

,path,wav2vec2,4gram,sentence
0,common_voice_bn_30991371.mp3,দেওয়ান ফরিদ গায়জী বাংলাদেশ আওয়ামী লীগের উপদেষ্...,দেওয়ান ফরিদ গাজী বাংলাদেশ আওয়ামী লীগের উপদেষ্ট...,দেওয়ান ফরিদ গাজী বাংলাদেশ আওয়ামী লীগের উপদেষ...
1,common_voice_bn_30991410.mp3,এর মাধ্যমে কারো গান গাওয়া দক্ষতাবা নিজস্ব ধর্ম...,এর মাধ্যমে কারও গান গাওয়া দক্ষতা বা নিজস্ব ধর্...,এর মাধ্যমে কারও গান গাওয়া দক্ষতা বা নিজস্ব ধর...
2,common_voice_bn_30991513.mp3,ডিজিটাল বিশ্বে তথ্য অন্য যে কোনো ডিজিটাল ফাইলে...,ডিজিটাল বিশ্বে তথ্য অন্য যে কোনো ডিজিটাল ফাইলে...,ডিজিটাল বিশ্বে তথ্য অন্য যে কোনও ডিজিটাল ফাইলে...
3,common_voice_bn_30991535.mp3,হিসাব পাওয়ার পর তিনি স্বল্প সময়ের মধ্যেই সম্পূ...,হিসাব পাওয়ার পর তিনি স্বল্প সময়ের মধ্যেই সম্পূ...,হিসাব পাওয়ার পর তিনি স্বল্প সময়ের মধ্যেই সম্...
4,common_voice_bn_30991592.mp3,নদীর পানিতে বয়ে আসা পলি দলদেশে জমে মাঝে মধ্যে ...,নদীর পানিতে বয়ে আসা পলি তলদেশে জমে মাঝে মধ্যে ...,নদীর পানিতে বয়ে আসা পলি তলদেশে জমে মাঝেমধ্যে ...
...,...,...,...,...
100931,common_voice_bn_31459116.mp3,তিনি বগুড়ার কাঠনাপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাটনাপাড়ায় জন্মগ্রহণ করেন।
100932,common_voice_bn_31459130.mp3,তবে ছোমান্টিজ থেকে বেশে থাকতে আমাদের কিছু কর ন...,তবে সমান টিজ থেকে বেশে থাকতে আমাদের কিছু করণীয়...,তবে জন্ডিস থেকে বেঁচে থাকতে আমাদের কিছু করণীয়...
100933,common_voice_bn_31459161.mp3,তিনি অসম উপত্যাকার সািত্য পুরস্কার ও সাহিত্য এ...,তিনি অসম উপত্যাকার সাহিত্য পুরস্কার ও সাহিত্য ...,তিনি অসম উপত্যকা সাহিত্য পুরস্কার ও সাহিত্য অক...
100934,common_voice_bn_31637719.mp3,টি পূর্বে অনেক মুচু ছিল,এটি পূর্বে অনেক উঁচু ছিল,এটি পূর্বে অনেক উঁচু ছিলো।


# Open_SLR

In [13]:
df_slr_1 = pd.read_csv('asr_bangla/data/slr_1_60.csv')
df_slr_2 = pd.read_csv('asr_bangla/data/slr_150_200.csv')
df_slr_3 = pd.read_csv('asr_bangla/data/slr_60_70.csv')
df_slr_4 = pd.read_csv('asr_bangla/data/slr_70_80.csv')
df_slr_5 = pd.read_csv('asr_bangla/data/slr_80_90.csv')
df_slr_6 = pd.read_csv('asr_bangla/data/slr_90_100.csv')

In [14]:
df_slr_1 = df_slr_1.append([df_slr_2,df_slr_3,df_slr_4,df_slr_5,df_slr_6])
df_slr_1 = df_slr_1.reset_index(drop=True)

In [15]:
df_slr_1['path'] = df_slr_1.path.apply(lambda x: os.path.basename(str(x)))

In [16]:
df_slr_1

,path,wav2vec2,4gram
0,000020a912.flac,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশের দায়িত্ব নেবে
1,000039928e.flac,এই ধরনের কাজ নিয়ে,এই ধরনের কাজ নিয়ে
2,00005debc7.flac,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ
3,00009e687c.flac,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে
4,00012843bc.flac,শাক দেশ গুরুতে,শাক দেশ গুড়িতে
...,...,...,...
149995,74c6f3b531.flac,একটা বরো কাজ হয়েছে,একটা বড় কাজ হয়েছে
149996,74c702e2d5.flac,জুন্ খালার সোনার দুল,জুন খেলার সোনার দুল
149997,74c706e710.flac,এরপরও বাংলা দেশে,এরপরও বাংলাদেশে
149998,74c70ce063.flac,ছাব্বিশ হাজার নয়শল ছাব্বিশ জনের,ছাব্বিশ হাজার নয় শ ছাব্বিশ জনের


In [17]:
df_slr = pd.read_csv('slr_train.csv')

In [18]:
df_slr['path'] = df_slr.path.apply(lambda x :os.path.basename(str(x)))

In [19]:
df_slr = df_slr_1.merge(df_slr,on='path',how='inner')

In [20]:
df_slr['path'] = 'slr_' + df_slr.path

In [21]:
df_slr

,path,wav2vec2,4gram,sentence
0,slr_000020a912.flac,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশে দায়িত্ব নেবে
1,slr_000039928e.flac,এই ধরনের কাজ নিয়ে,এই ধরনের কাজ নিয়ে,এ ধরণের কার্ড নিয়ে
2,slr_00005debc7.flac,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ
3,slr_00009e687c.flac,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে,হাসির বিষয় হয়েই আছে
4,slr_00012843bc.flac,শাক দেশ গুরুতে,শাক দেশ গুড়িতে,সার্ক দেশগুলোতে
...,...,...,...,...
149995,slr_74c6f3b531.flac,একটা বরো কাজ হয়েছে,একটা বড় কাজ হয়েছে,একটা বড় কাজ হয়েছে
149996,slr_74c702e2d5.flac,জুন্ খালার সোনার দুল,জুন খেলার সোনার দুল,ঝুনুখালার সোনার দুল
149997,slr_74c706e710.flac,এরপরও বাংলা দেশে,এরপরও বাংলাদেশে,এর পরও বাংলাদেশে
149998,slr_74c70ce063.flac,ছাব্বিশ হাজার নয়শল ছাব্বিশ জনের,ছাব্বিশ হাজার নয় শ ছাব্বিশ জনের,২৬ হাজার ৯শ ২৬ জনের


# Data_Merged

In [22]:
df_actual = df_slr.append(df_cv)

In [23]:
df_actual = df_actual.dropna()

In [24]:
df_actual = df_actual.reset_index(drop=True)

In [25]:
df_actual.rename(columns = {'4gram':'arpa_4gram'}, inplace = True)

In [26]:
df_actual

,path,wav2vec2,arpa_4gram,sentence
0,slr_000020a912.flac,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশে দায়িত্ব নেবে
1,slr_000039928e.flac,এই ধরনের কাজ নিয়ে,এই ধরনের কাজ নিয়ে,এ ধরণের কার্ড নিয়ে
2,slr_00005debc7.flac,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ
3,slr_00009e687c.flac,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে,হাসির বিষয় হয়েই আছে
4,slr_00012843bc.flac,শাক দেশ গুরুতে,শাক দেশ গুড়িতে,সার্ক দেশগুলোতে
...,...,...,...,...
248612,common_voice_bn_31459116.mp3,তিনি বগুড়ার কাঠনাপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাটনাপাড়ায় জন্মগ্রহণ করেন।
248613,common_voice_bn_31459130.mp3,তবে ছোমান্টিজ থেকে বেশে থাকতে আমাদের কিছু কর ন...,তবে সমান টিজ থেকে বেশে থাকতে আমাদের কিছু করণীয়...,তবে জন্ডিস থেকে বেঁচে থাকতে আমাদের কিছু করণীয়...
248614,common_voice_bn_31459161.mp3,তিনি অসম উপত্যাকার সািত্য পুরস্কার ও সাহিত্য এ...,তিনি অসম উপত্যাকার সাহিত্য পুরস্কার ও সাহিত্য ...,তিনি অসম উপত্যকা সাহিত্য পুরস্কার ও সাহিত্য অক...
248615,common_voice_bn_31637719.mp3,টি পূর্বে অনেক মুচু ছিল,এটি পূর্বে অনেক উঁচু ছিল,এটি পূর্বে অনেক উঁচু ছিলো।


In [27]:
df_actual['sentence'] = df_actual['sentence'].parallel_apply(lambda x : remove_punctuations(x))
df_actual["sentence"]=df_actual["sentence"].parallel_apply(lambda x:normalize(x))

In [28]:
#df_actual.to_csv('df_data_actual.csv',index=False)

In [29]:
df_actual['wav2vec2_label'] = df_actual.parallel_apply(lambda x : error_label(x.sentence,x.wav2vec2),axis=1)

In [30]:
df_actual['arpa_label'] = df_actual.parallel_apply(lambda x : error_label(x.sentence,x.arpa_4gram),axis=1)

In [32]:
df_actual['wer_wav2vec2'] = df_actual.wav2vec2_label.parallel_apply(lambda x: wer_calc(x))

In [33]:
df_actual['wer_arpa'] = df_actual.arpa_label.parallel_apply(lambda x: wer_calc(x))

In [34]:
df_actual.to_csv('asr_bangla/data/asr_cv_slr.csv',index=False)

In [ ]:
df_actual[(df_actual.wer_wav2vec2 >= 30) & (df_actual.wer_wav2vec2 <= 90)]

# Custom_data_prothom_alo

In [ ]:
sym_spell = SymSpell(max_dictionary_edit_distance=5, prefix_length=7)
dictionary_path = 'asr_bangla/data/prothom_alo_word_freq.txt'
sym_spell.load_dictionary(dictionary_path, 0, 1,separator=" ")

In [ ]:
df_prothom_alo = pd.read_csv('bangla_text_data.csv')

df_prothom_alo= df_prothom_alo.dropna(how='any')

In [ ]:
def get_len(sen):
    sen = sen.split()
    l = len(sen)
    return l

In [ ]:
df_prothom_alo = df_prothom_alo.sample(300000)

In [ ]:
df_prothom_alo['lena'] = df_prothom_alo['sentence'].apply(lambda x: get_len(str(x)) )

In [ ]:
df_prothom_alo.lena.value_counts()

In [ ]:
df_prothom_alo = df_prothom_alo[(df_prothom_alo.lena >= 3) & (df_prothom_alo.lena <= 15)]

In [ ]:
df_prothom_alo = df_prothom_alo.sample(100000)

In [ ]:
df_prothom_alo.lena.value_counts()

In [ ]:
df_prothom_alo.to_csv('custom_data_for_bert.csv',index=False)

In [ ]:
df_prothom_alo = pd.read_csv('custom_data_for_bert.csv')

In [ ]:
df_prothom_alo

In [ ]:
character = ['অ','আ','ই','ঈ','উ','ঊ','এ','ঐ','ও','ঔ','ঋ','ঃ','ং','ৎ'
             'ক','খ','গ','ঘ','ঙ','চ', 'ছ','য','জ', 'ঝ','ট','ঠ','ড','র','ড়','ঢ়',
             'ণ','ত','থ','দ','ধ','ন','প','ফ','ব','ভ','ম','স','শ','ষ','য়','হ']

In [ ]:
JuktakkhorList = [' ক্ট ' , ' ক্ক ' , ' ক্ত ' , ' ক্য ' , ' ক্র ' , ' ক্ল ' , ' ক্ষ ' , ' ক্ষ্ণ ' , ' ক্ষ্ম ' , ' ক্ষ্য ' , ' ক্স ' , ' খ্র ' , ' গ্ধ ' , ' গ্ধ্য ' , ' গ্ন ' , ' গ্ন্য ' , ' গ্ব ' , ' গ্র ' , ' গ্র্য ' , ' গ্ল ' , ' ঘ্ন ' , ' ঘ্র ' , ' ঙ্ক্য ' , ' ঙ্গ্য ' , ' চ্চ ' , ' চ্ছ্ব ' , ' চ্য ' , ' জ্জ ' , ' জ্জ্ব ' , ' ট্ট ' , ' জ্ব ' , ' জ্য ' , ' জ্র ' , ' ট্য ' , ' ট্র ' , ' ড্ড ' , ' ড্র ' , ' ণ্ট ' , ' ণ্ঠ ' , ' ণ্ড ' , ' ণ্ণ ' , ' ণ্য ' , ' ৎক ' , ' ৎখ ' , ' ত্ত ' , ' ত্ত্ব ' , ' ত্ত্য ' , ' ত্ন ' , ' ৎপ ' , ' ত্ব ' , ' ত্ম ' , ' ত্ম্য ' , ' ত্য ' , ' ত্র ' , ' ত্র্য ' , ' ৎস ' , ' দ্ঘ ' , ' দ্দ ' , ' দ্ধ ' , ' দ্ব ' , ' দ্ভ ' , ' দ্ভ্র ' , ' দ্ম ' , ' দ্য ' , ' দ্র ' , ' দ্র্য ' , ' ধ্ব ' , ' ধ্য ' , ' ধ্র ' , ' ন্ট ' , ' ন্ট্র ' , ' ন্ঠ ' , ' ন্ড ' , ' ন্ড্র ' , ' ন্ত ' , ' ন্ত্ব ' , ' ন্ত্য ' , ' ন্ত্র ' , ' ন্ত্র্য ' , ' ন্থ ' , ' ন্দ ' , ' ন্দ্ব ' , ' ন্দ্র ' , ' ন্ধ ' , ' ন্ন ' , ' ন্য ' , ' প্ট ' , ' প্ত ' , ' প্ন ' , ' প্প ' , ' প্য ' , ' ব্দ ' , ' ব্ধ ' , ' ব্ব ' , ' ব্র ' , ' ভ্য ' , ' ভ্র ' , ' ম্প্র ' , ' ম্ব ' , ' ম্ম ' , ' ম্য ' , ' ম্র ' , ' য্য ' , ' র্ক ' , ' র্গ্য ' , ' র্ঘ্য ' , ' র্জ্য ' , ' র্থ্য ' , ' র্ব্য ' , ' র্খ ' , ' র্গ ' , ' র্ঘ ' , ' র্চ ' , ' র্ছ ' , ' র্জ ' , ' র্ঝ ' , ' র্ট ' , ' র্ড ' , ' র্ণ ' , ' র্ত ' , ' র্থ ' , ' র্দ ' , ' র্দ্ব ' , ' র্দ্র ' , ' র্ধ ' , ' র্ধ্ব ' , ' ল্ট ' , ' ল্ড ' , ' ল্প ' , ' ল্ম ' , ' ল্য ' , ' ল্ল ' , ' শ্ছ ' , ' শ্ন ' , ' শ্ব ' , ' শ্র ' , ' শ্ল ' , ' ষ্ক ' , ' ষ্ক্র ' , ' ষ্ট ' , ' ষ্ট্য ' , ' ষ্ট্র ' , ' ষ্ঠ ' , ' ষ্প ' , ' স্ট ' , ' স্ট্র ' , ' স্ত্র ' , ' স্ত ' , ' স্থ্য ' , ' স্য ' , ' স্র ' , ' স্ল ' , ' হ্র ' , ' হ্ল ' , ' গ্ম ' , ' ম্ভ ' , ' স্ম ' , ' ঙ্ঘ ' , ' ঙ্ক্ষ ' , ' ঙ্খ ' , ' ঙ্ক ' , ' ঙ্‌ক্ত ' , ' ঞ্চ ' , ' ঞ্ছ ' , ' ঞ্জ ' , ' হ্ন ' , ' ঙ্গ ' , ' জ্ঞ ' , ' গ্য ' , ' চ্ছ '] 

In [ ]:
ReplaceDict = {
      'ক' : ['ল','য'],
      'খ' : ['কগ','কজ','লহ','ঝ'],
      'গ' : ['ফ','হ'],
      'ঘ' : ['ফগ','হজ'],
      'ঙ' : ['ব','ম'],
      'চ' : ['ভ','চজ','চগ','ভহ'],
      'ছ' : ['ভ','চজ','চগ','ভহ'],
      'জ' : ['ক','হ'],
      'ঝ' : ['কজ','হগ'],
      'ট' : ['র'],
      'ঠ' : ['তজ','তগ','রহ'],
      'ড' : ['স','ফ'],
      'ঢ' : ['দজ','দ্গ','শ','ফহ'],
      'ণ' : ['ব','ম'],
      'ত' : ['র'],
      'থ' : ['তজ', 'তগ', 'রহ'],
      'দ' : ['স', 'ফ'],
      'ধ' : ['দজ', 'দ্গ', 'শ' ,'ফহ'],
      'ন' : ['ব','ম'],
      'প' : ['ও',' ো'],
      'ফ' : ['দ','গ'],
      'ব' : ['ভ','ন'],
      'ভ' : ['‍ব','চ'],
      'ম' : ['ন'],
      'য' : ['হ','ক'],
      'র' : ['এ',' ে','ত'],
      'ল' : ['ক'],
      'শ' : ['সজ','সগ','আহ','ঢ'],
      'ষ' : ['সজ','সগ','আহ','ঢ'],
      'স' : ['আ',' া','দ'],
      'হ' : ['গ','য'],
      'য়' : ['ত','উ','ু'],
      'ড়' : ['এ',' ে','ত'],
      'ঢ়' : ['এ',' ে','ত'],
      'ৎ' : ['র'],
      'ং' : ['ব','ম'],
      'ঃ' : ['গ','য'],
      'অ' : ['প',' ি','ই'],
      'আ' : ['স'],
      'ই' : ['উ','অ'],
      'ঈ' : ['উ','অ'],
      'উ' : ['ই',' ি'],
      'ঊ' : ['ই',' ি'],
      'ঋ' : [''],
      'এ' : ['ও','র'],
      'ঐ' : [''],
      'ও' : ['প','ই'],
      'ঔ' : [''],
      'া' : ['স'],
      'ি' : ['উ','অ'],
      'ো' : ['প',' ি'],
      'ৌ' : [''],
      'ে' : ['ো','র'],
      'ৈ' : ['']
    }

In [ ]:
SameClusterDict = {
    'অ' : ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'আ' : ['আ','অ্যাঁ','আঃ','ওয়া','এ'],
    'ই':['ই','ঈ','এ','ে','য়'],
    'ঈ':['ই','ঈ','এ','ে','য়'],
    'উ' : ['উ''উঁ','ঊ','ু'],
    'ঊ' : ['উ''উঁ','ঊ','ু'],
    'ঋ' : ['রি','হ্রী','ড়ি','ঢ়ি','ব্রই','ৃ','হ্র'],
    'এ' : ['এ','অ্যা','আঃ'],
    'ঐ': ['অই','ঈ','ওই' , 'ই', 'অ্যাই'],
    'ও': ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ঔ': [ 'ও','আ','অউ','অ্যা','া','ওঁ','ওঃ'],
    'ক': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'খ': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'গ': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'ঘ ': ['খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ,'ঘো ','ঘও'],
    'ঙ': ['◌ং','ও','আ','অ্যা','া','ওঁ','ওঃ','অং'],
    'চ': ['চ','ও','আ','অ্যা','া','ওঁ','ওঃ','চোঁ','চও','ছোঁ','ছো','ছও','ছ'],
    'ছ': ['চ','ও','আ','অ্যা','া','ওঁ','ওঃ','চোঁ','চও','ছোঁ','ছো','ছও','ছ'],
    'জ': ['জ','য','জো','যো','ঝ','ঝও'],
    'ঝ': ['জ','য','জো','যো','ঝ','ঝও'],
    'ঞ': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'ট' : ['ত','তঁ','তোঁ','টয়','তো','ট'],
    'ঠ' : ['থ','ত','তঁ','তোঁ','টয়','তো','ট'],
    'ড' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ঢ': ['দ','দো','দ্য','ডঃ','দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ণ': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'ত' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'থ': ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'দ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ধ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ন': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'প' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ফ' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ব' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ভ' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ম' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'য' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'র' : ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    'ল' : ['ল','লয়','লো'],
    'শ' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    'ষ' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    'স' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    '‍হ': ['হ','ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ড়' : ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    '‍ঢ়': ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    '‍য় ': ['ই','ঈ','এ','ে','য়'],

    'ৎ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ং' : ['◌ং','ও','আ','অ্যা','া','ওঁ','ওঃ','অং'],
    'ঃ': ['হ','ও','আ','অ্যা','া','ওঁ','ওঃ'],
    '‍ঁ' : [''],

    '‍ি': ['ই','ঈ','এ','ে','য়'],
    'ী' : ['ই','ঈ','এ','ে','য়'],
    'ে':['এ','অ্যা','আঃ'],
    'ৈ'  :['এ','অ্যা','আঃ'],
    'ো' : ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ৌ':['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ৃ' : ['রি','হ্রী','ড়ি','ঢ়ি','ব্রই','ৃ','হ্র'],
     '◌ূ'  : ['উ''উঁ','ঊ','ু'],
     ' ু'  : ['উ''উঁ','ঊ','ু']
}

In [ ]:
def apply_error(string,error):
    random.seed(time.time()+200)
    errors = ['Cluster','Replace','Juktakkhor','Character','Deletion','Noerror']
    if error == "Cluster":
        if string in list(SameClusterDict.keys()):
            string = random.choice(SameClusterDict[string])
    if error == "Replace":
        if string in list(ReplaceDict.keys()):
            string = random.choice(ReplaceDict[string])
    if error == "Juktakkhor":
        string = random.choice(JuktakkhorList)
    if error == "Character":
        string = random.choice(character)
    if error == "Deletion":
        string = ""
    if error == "insertion":
        temp_error = random.choice(errors)
        string = string + apply_error(string,temp_error)
    if error == "Noerror":
        pass
    return string

In [ ]:
def error_generator(word):
    errors = ['Cluster','Replace','Juktakkhor','Character','Deletion','Insertion','Noerror']
    string = []
    string[:0] = word
    random.seed(time.time())
    noise_ratio = random.randint(0,int(len(word)*0.40))
    print(noise_ratio)
    for n in range(noise_ratio):
        for i in range(len(string)):
            random.seed(time.time()+2)
            error_t = random.random()
            if error_t >=0.5:
                random.seed(time.time()+100)
                error = random.choice(errors)
                string[i] = apply_error(string[i],error)
    word = "".join(string)
    word = word.replace(" ", "")
    return word

In [ ]:
word = 'অজপারাগায়ে'

In [ ]:
error_generator(word)

In [ ]:
sentence = 
df_prothom_alo.sentence.tolist()

In [ ]:
df_custom_error_dataset = pd.DataFrame()
for sen in tqdm(sentence):
    sent = sen
    sen = sen.split()
    sent = sent.split()
    random.seed(time.time()+356)
    random_index = random.randint(0,len(sen)-1)
    sen[random_index] = '[MASK]'
    error_word = error_generator(sent[random_index])
    temp_dict = {'sentence':" ".join(sent),'masked_sen':" ".join(sen),'error_word':error_word }
    df_custom_error_dataset = df_custom_error_dataset.append(temp_dict,ignore_index=True)

In [ ]:
df_custom_error_dataset.sample(5)

In [ ]:
df_custom_error_dataset.to_csv('custom_error_dataset.csv',index=False)